In [ ]:
import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
try:
    import fastbook
    print(f"fastbook version: {fastbook.__version__}")
except ImportError:
    print("fastbook is not installed.")

In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [ ]:
#hide 
!pip install -Uqq fastai

In [ ]:

path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
#hide
Path.BASE_PATH = path

In [ ]:


path.ls()

In [ ]:
(path/'train').ls()

In [ ]:
(path/'valid').ls()

In [ ]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
sevens

In [ ]:
im3 = Image.open(threes[1])
im3

In [ ]:
array(im3)[4:10,4:10]

In [ ]:
tensor(im3)[4:10,4:10]

array and tensor allows us to see an image represented as an array. [4:10,4:10] allow us to see the lets use see rows 4 - 10 and columns 4-10. 2d array, so row is len(array), column is len(array[0])

In [ ]:
im3_t = tensor(im3)
df = pd.DataFrame(im3_t[4:15,4:22])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

**The entire image contains 28 pixels across and 28 pixels down, for a total of 784 pixels. This is much smaller than an image that you would get from a phone camera whihc has millions of pixels**

Numpy array: each number in the array represents the intensity or color value of a pizel in the image. The interpretation of these values depends on the type of image.

Grayscale Images(used in the example above): in grayscale image,s each number typically represents the intensity of the pixel. This can be a single value ranging form 0 to 255 in an 8-bit image, where 0 is black and 255 is white

Color images (RGB): In color images, each pixel is often represented by three values, corresponding to the red, green and blue color channels. each channel value can range from 0 to 255, where 0 is no intensity and 255 is full intensity for that color. combining these three channels produces a wide range of colors

Color Images (RGBA) : In some cases, especially when dealing with transparencty, an additional channel is used, known as the alpha channel (RGBA). the alpha channel represents the transparency of the pixel, with 0 being fully transparent and 255 being fully opaque.

**Now how might a computer might be able to recognize these two differetn digits. **

A simple approach would be to get the average of pixel values for each of our two groups.

In [ ]:
seven_tensors = [tensor(Image.open(num)) for num in sevens]
# storing a bunch of 2d arrays into an array 
three_tensors = [tensor(Image.open(num)) for num in threes]

len(seven_tensors), len(three_tensors)

In [ ]:
len(seven_tensors[0]), len(seven_tensors[0][0])
28*28

each tensor would have around 800 pixels, so sum of every pixel / 784 

TO get started, we need to combine all the images in the tensor lists into a single three-dimensional tensor.

In [ ]:
stack_sevens = torch.stack(seven_tensors).float()/255
stack_threes = torch.stack(three_tensors).float()/255

The most important attribute of a tensor is its shape. This tells you the length of each axis. 

**rank:** the number of axes or dimensions in a tensor

**shape:** the size of each axis of a tensoe

In [ ]:
mean3 = stack_threes.mean(0)
show_image(mean3)

this is the ideal number 3 based on all the numpy numbers gathered from the 7k handwritten 3s

In [ ]:
mean7 = stack_sevens.mean(0)
show_image(mean7)

###

How can we determine its distasnce from our ideal 3 or 7? 
Should we just add up the differences between the pixels of this image and the ideal digit? 
Some differences will be positive while others will be negative since some areas with be 0 whiles others are darkened
**How can we avoid this**

**2 ways to measure distance in this context**

1. take the mean of the absolute value of differences . absolute value is the function that replaces negative values with positive values. ** Called the mean absolute difference or L1 norm
2. take the mean of the square of differences (which makes everything positive) and then take the square root (which undoes the squaring). this i called the ** root mean squared error (RMSE) or L2 norm

In [ ]:
a_3 = stack_threes[1]
dist_3_abs = (a_3 - mean3).abs().mean()
dist_3_sqr = ((a_3-mean3)**2).mean().sqrt()
dist_3_abs, dist_3_sqr

In [ ]:
dist_7_abs = (a_3 - mean7).abs().mean()
dist_7_sqr = ((a_3 - mean7)**2).mean().sqrt()
dist_7_abs,dist_7_sqr

as you can see from the results, the 3 is closer to the ideal 3 than the ideal 7. SO if we were to determine if am_3 is a 3 based on its distance form the "ideal" 3 or 7, we would say it is a 3

**Computing Metrics Using Broadcasting**

metric: a number that is calculated based on the predictions of our models, and the correct labels in our dataset, in order to tell us how good our model is

**we want to calculate our metric over a validation set to avoid overfitting**

**Stochastic Gradient Descent (SGD)**

All the testing we did above is not deep learning. There are no parameters, weight assiignment, or any way of improving based on testing the effectiveness of a weight assessment. 

pixel similarity approach does not do this. 

Instead of trying to find the similarity between an image and an ideal image (average pixel count of a bunch of images), we could instead look at each individual pixel and come up with a set of weights for each one, suc that thehighest weights are associated with those pixels most likely to be black for a particular category. 

**Example**
pixels toward the bottom right are not very likely to be activated for a 7, so they should have a low weight for a 7, but they are likely to be acttivated fro an 8, so they should have a high weight for an 8.. This can be represented as a function and aset of weight values for each possible category 

def pr_eight(x, w): return (x*w).sum() 
x: image, represented as a vector, 2d array where each index represents a row of the image 
w: weight as a vector. 

for this function, we need a way to update the weights to make them a lttle bit better. With such an approach, we can repeat that step a number of times, making the weights better and better, until they are as good as we can make them. 

**we want to find the specific values for the vector w that causes the result of our function to be high for those images that are actually 8s, and low for those image that are not.** 

think of it as seraching for the best vector w is the way to search for the best funciton for recognising 8s

Steps to turn the function abover into a machine learning classifier:

1. Initilize the weights/
2. For each image use these wegiihts to predict whether it appears to be a 3 or a 7.
3. based on these predictions calculate how good the model is (its loss). 
4. calculate the graident, whihc emasures for each weight, how changing that weight would change the loss
5. Step (that is, change all the weights based on that calculation
6. go bac to step 2, and repeat the process
7. iterate until you decide to stop the training process ( for instance, because the mmodel is good enough or you dont want to wait any longer)

**the steps above are the key to the training of all depp learning models. the dep learning turns out to reply entirely on these steps is extremely surprising and counterintuitive.**

![image.png](attachment:13289791-0bf8-4c3f-944d-f5af957cae62.png)

Basic principles for those 7 steps 
* Initialize:: WE initialize the parameters to random values. They are gonna be changed so the initial values do not matter 
* Loss:: this what Samuel referred to when he spoke of testing the effectiveness of any current weight assignment in terms of actual performance. 
       * we neeed a function that will return a number that is small if the performance of the model is good
* Step:: A simple way to figure out whether a weight should be increased a bit or decreased a bit, would be just to try it: increase the weight by a small amount, and see if the loss goes up or down.
    * the magic of calculas allows us to figure out which direction an dby roughly how much, to change each weight, without having to try all these small changes. The way to do this is by calculating gradients. this is just a performance optiumization
* Stop:: once we've decided how many epochs to trian the model for, we apply that decision.

**Calculating Gradients**
calculus allows us to quickly calculate whether our loss will go up or down when we adjust our parameters up or down. In other words, the gradients will tell us how much we have to change each weight to make our model better

**Derivative review**
Derivative the the function that calculates the change. For our gradient, the derivative determines how our loss changes as we increase or decrease the parameter

**Key to macine learning: having a way to change the paremeters of a fucntion to make it smaller. Calculus procudes us a computationa shortcut, the derivative, whihc lets us directly calcualte the gradients of our functions.**

! importnat thing to be aware of is that our function has lots of weights that we need to adjust, so when we calculat ethe deriative we wont get back one number, but lots of them , a gradient for every weight. 
Solution is to calculat ethe derivative with respect to one weight and treat all the other ones as constant, and then repeat that for each other weight.

pytorch: automatically compute the derivative of nearly every function. Does it really fast as well

In [ ]:

data = [[1,2,3],[4,5,6]]
arr = array (data)
tns = tensor(data)
tns

In [ ]:
def f(x): return x**2

In [ ]:
xt = tensor(3.).requires_grad_()
xt

In [ ]:
yt = f(xt)
yt

In [ ]:

yt.backward()

In [ ]:
xt.grad

**Stepping With a Learning Rate**

Deciding how ot change our parameters based on the values of the gradients is an important part of the deep learning process. 
All approaches start with the basic idea of multiplying the gradient by some small number, called the learning rate (LR)
Learning rate: often a number between 0.001 and .1.

once a learning rate is determined, you can adjust the parameters using this simple funciton:

**w -= gradient(w) * lr**

this is known as stepping your parameters, using an optimizer step

Notice how we subtract the graident * lr form the paramter to update it. 
Think about how this would allow us to have th smallest loss possible. 
if the slope is negative, then this would increase it since the gradiant will be negative or if the slope if positive this would allow us to decreaes the parameter

**we want to adjust our parameters in the direction of the slope because our goal in deep learning is to minimize the loss**

**if the learning rate is too late: we will have to do a lot of steps:
![image.png](attachment:52438202-c613-4ae4-875b-1a7b1d7b928f.png)

**Picking a learning rate that's too high is even worse- it can result in the loss getting worse**
![image.png](attachment:ce2deb1d-c131-4468-9b41-9defd2d76b9f.png)

**AN END-TO-END SGD Example**

In [ ]:
time = torch.arange(0,20).float(); time
## this give use a tensor of floats from 0 -20